# Code Snippets

Little code samples

In [1]:
import pandas as pd

### Add a Column to DataFrame

Add a column of to a DataFrame object. We are going to use pd.concat() function to achieve the result. To make it work, we need to:

1. Create a dataframe of a single column, with the column name specified by the user;
2. Align the index of the new dataframe with the dataframe we are appending to;
3. Call concat().

#### Add a List of Fixed Values As a Column

In [17]:
from itertools import repeat

"""
    Add a column of fixed value to a dataframe
    
    [String] column (column name), [Object] value (any value type that can be put into a dataframe), [DataFrame] df
        => [DataFrame] df (new DataFrame with a column appended)
"""
addColumn = lambda column, value, df: \
    pd.concat( [ df
               , pd.DataFrame(repeat([value], len(df)), columns=[column], index=df.index)
               ]
             , axis=1
             )

Try it out

In [18]:
df = pd.DataFrame( [[88, 3], [90, 1], [75, 10]]
                 , columns=['Score', 'Rank']
                 , index=['Isaac', 'Yan', 'Chuyi'])
df

,Score,Rank
Isaac,88,3
Yan,90,1
Chuyi,75,10


In [19]:
addColumn('Date', '2020-05-16', df)

,Score,Rank,Date
Isaac,88,3,2020-05-16
Yan,90,1,2020-05-16
Chuyi,75,10,2020-05-16


#### Add a Series Object As a Column

In [53]:
"""
    Add a Series Object as a column to a dataframe
    
    The index of the series object is not important but its length must be the same as the length 
    of the dataframe to be appended, otherwise an exception will be thrown.
    
    [String] column (column name), [Series] s, [DataFrame] df
        => [DataFrame] df (new DataFrame with series s as a column appended)
"""
addSeriesColumn = lambda column, s, df: \
    pd.concat( [ df
               , pd.DataFrame({column: s.values}, index=df.index)
               ]
             , axis=1
             )

In [54]:
dates = pd.Series(['2020-05-16', '2020-05-17', '2020-05-18'])
dates

0    2020-05-16
1    2020-05-17
2    2020-05-18
dtype: object

In [55]:
addSeriesColumn('Date', dates, df)

,Score,Rank,Date
Isaac,88,3,2020-05-16
Yan,90,1,2020-05-17
Chuyi,75,10,2020-05-18


In [52]:
# Will go wrong! Because the length of the dataframe is not equal to the length of the series
# addSeriesColumn( 'Date'
#                , pd.Series(['2020-05-16', '2020-05-17'])
#                , df)